In [55]:
A = """Authorised capital of the company as on the date of filing
Number of members of the company as on the date of filing
100,000
English
Hindi
0
Note- 
-All fields marked in * are to be mandatorily filled. 
-Figures appearing in the e-Form should be entered in Absolute Rupees only. Figures should not be rounded off if any 
other unit like hundreds, thousands, lakhs, millions or crores.
SEGMENT- I: INFORMATION AND PARTICULARS IN RESPECT OF BALANCE SHEET
    
I. General information of the company
    (b) Global location number (GLN) of company 
Pre-Fill
U29248GJ2009PTC057637
1. (a) *Corporate identity number (CIN) of company  
officeclient1@rediffmail.com
(c) *e-mail id of the company  
C/O. MAHAVIR SALES CORPORATION, SUR. NO. 269, 
NR SHIVKRUPA WEIGH,JILLA GARDEN MAIN RD,BAPUNAGAR, 
RAJKOT 
Gujarat 
INDIA 
(b) Address of the registered      
INGENIOUS CONSTRUCTION EQUIPMENTS PRIVATE LIMITED
2. (a) Name of the company
3. * Financial year to which financial statements relates
From
01/04/2014
(DD/MM/YYYY)
(DD/MM/YYYY)
31/03/2015
To
4. (a)* Date of Board of Directors’ meeting in which financial statements are approved
(DD/MM/YYYY)
25/09/2015
office of the company  """

partitions = re.split(r'Text From \w+', A)
            
            # Define the desired columns to extract
desired_columns = ['Corporate identity number (CIN) of company']
            
            # Extract the data from the text
result = []
for partition in partitions:
    partition_data = {}
    for column in desired_columns:
        match = re.search(fr"([\w\d]+)[\n\s]*.*{re.escape(column)}", partition)
        if match:
            partition_data[column] = match.group(1).strip()
    result.append(partition_data)
print(result)


[{'Corporate identity number (CIN) of company': 'U29248GJ2009PTC057637'}]


In [76]:
!pip install PyPDF2
import os
import re
import pandas as pd
from PyPDF2 import PdfReader

folder_path = r'C:\Users\Priyank\Downloads\Client data'  # Replace with your folder path

file_list = os.listdir(folder_path)

result = []  # Variable to store the extracted data

for file_name in file_list:
    if file_name.endswith('.pdf'):  # Process only PDF files
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'rb') as file:
            pdf = PdfReader(file)
            num_pages = len(pdf.pages)
            text = ""
            for page_num in range(num_pages):
                page = pdf.pages[page_num]
                text += page.extract_text()
                
            partitions = re.split(r'Text From \w+', text)
            
            # Define the desired columns to extract
            desired_columns = ['Authorised capital of the company as on the date of filing','Corporate identity number (CIN) of company','e-mail id of the company']
            data_list = []
            # Extract the data from the text
            result = []
            for partition in partitions:
                partition_data = {}
                for column in desired_columns:
                    match = re.search(fr'{column}\n.*?([\d,]+)', partition)
                    match1 = re.search(fr"(\w+@\w+(?:\.\w+)+)[\n]*.*{re.escape(column)}", partition)
                    match2 = re.search(fr"([\w]+[\d]+)[\n]*.*{re.escape(column)}", partition)
                    if match:
                        partition_data[column] = match.group(1).strip()
                    elif match1:
                        partition_data[column] = match1.group(1).strip()
                    elif match2:
                        partition_data[column] = match2.group(1).strip()
                result.append(partition_data)
        data_list.extend(result)

# Create DataFrame from the extracted data
df = pd.DataFrame(data_list)

df

,Authorised capital of the company as on the date of filing,Corporate identity number (CIN) of company,e-mail id of the company
0,"100,000",U29248GJ2009PTC057637,officeclient1@rediffmail.com


In [52]:
!pip install PyPDF2
import os
import re
from PyPDF2 import PdfReader

folder_path = r'C:\Users\Priyank\Downloads\Client data'  # Replace with your folder path

file_list = os.listdir(folder_path)

result = []  # Variable to store the extracted data

for file_name in file_list:
    if file_name.endswith('.pdf'):  # Process only PDF files
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'rb') as file:
            pdf = PdfReader(file)
            num_pages = len(pdf.pages)
            text = ""
            for page_num in range(num_pages):
                page = pdf.pages[page_num]
                text += page.extract_text()
        print(text)

Please wait... 
  
If this message is not eventually replaced by the proper contents of the document, your PDF viewer may not be able to display this type of document.   You can upgrade to the latest version of Adobe Reader for Windows®, Mac, or Linux® by visiting  http://www.adobe.com/go/reader_download.   For more assistance with Adobe Reader visit  http://www.adobe.com/go/acrreader.   
Windows is either a registered trademark or a trademark of Microsoft Corporation in the United States and/or other countries. Mac is a trademark of Apple Inc., registered in the United States and other countries. Linux is the registered trademark of Linus Torvalds in the U.S. and other countries.
Please wait... 
  
If this message is not eventually replaced by the proper contents of the document, your PDF viewer may not be able to display this type of document.   You can upgrade to the latest version of Adobe Reader for Windows®, Mac, or Linux® by visiting  http://www.adobe.com/go/reader_download.   F